### Instructions for Running Notebook

Click Cell -> Run All. Click on local IP in final cell. Now you can view/use Dashboard.

In [1]:
# pip install dash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
# Load csv shot distribution data
all_teams_df = pd.read_csv('nba_shooting_data_19-20.csv')
# Load stylesheet for Dashboard
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [3]:
#formatting for chart
def clean_chart_format(fig):
    fig.update_layout(
        paper_bgcolor="white",
        plot_bgcolor="white",
        margin=dict(
            t=20
        )
    )
    fig.update_traces(marker=dict(line=dict(width=1, color='Navy')),
                      selector=dict(mode='markers'))
    fig.update_coloraxes(
        colorbar=dict(
            thicknessmode="pixels", thickness=15,
            outlinewidth=1,
            outlinecolor='#909090',
            lenmode="pixels", len=300,
            yanchor="top",
            y=1,
        ))
    fig.update_yaxes(showgrid=True, gridwidth=1, tickson='boundaries', gridcolor='LightGray', fixedrange=True)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray', fixedrange=True)
    return True

In [4]:
def makeshotchart(input_df, color_continuous_scale=None, size_col='shts_total', col_col='acc', range_color=None):
    max_bubble_size = 15
    if color_continuous_scale is None:
        color_continuous_scale = px.colors.diverging.RdYlBu_r
    if range_color is None:
        range_color = [min(input_df[col_col]), max(input_df[col_col])]

    fig = px.scatter(
        input_df, x='min_mid', y='player', size=size_col,
        color=col_col,
        color_continuous_scale=color_continuous_scale,
        range_color=range_color,
        range_x=[0, 49],
        range_y=[-1, len(input_df.player.unique())],
        hover_name='player', hover_data=['shts_perct', 'shts_freq'],
        render_mode='svg'
    )
    fig.update_coloraxes(colorbar=dict(title='Points per 100 possessions'))
    fig.update_traces(marker=dict(sizeref=2. * 30 / (max_bubble_size ** 2)))
    fig.update_yaxes(title="Players")
    fig.update_xaxes(title='Minutes Played (Divide by 12 for Quarters)', tickvals=list(range(0, 54, 12)))

    return fig

In [5]:
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

server = app.server

app.title = 'NBA Shooting Dashboard per Minute by Team'
team_names = all_teams_df.team.unique()
team_names.sort()
app.layout = html.Div([
    html.Div([
        dcc.Markdown(
            """
            ### Shot Frequency & Efficiency for the 2019-20 NBA Season
            
            ##### Use the dropdown to select a team, the default 'Leaders' shows leaders from each team across the league.

            This dashboard compares players using **shot frequency** (shts_freq) and **efficiency** (ppp) across a time series (48 minutes of an NBA game) using a bubble chart.
            
            The X axis is divided up into minutes, 48 minutes in a whole game. The Y axis are the players on each team and the size of the bubbles represents the frequency of shots taken.
            
            **Frequency**: Portion of shots taken by player relative to the rest of the team's shots, indicated by size. **Bigger** the bubble, the **more shots taken** during that time period. **Smaller** the bubble, the **less shots taken** during that time period.

            **Efficiency**: Points per 100 possessions (ppp), indicated by color. **More red** is **better**, **more blue** is **worse**. 
            Hovering over the bubbles also gives you the shooting percentage (shts_perct). There's typically a correlation between a high shooting percentage and a high pts per 100 possessions.

            Players with less than 1 percent of team shots are classified under 'Others'
            """
        ),
    ]),
    html.Div([
        dcc.Dropdown(
            id='group-select',
            options=[{'label': i, 'value': i} for i in team_names],
            value='Leaders',
            style={'width': '140px'}
        )
    ]),
    dcc.Graph(
        'bubble-chart',
        config={'displayModeBar': False}
    )
])

In [6]:
@app.callback(
    Output('bubble-chart', 'figure'),
    [Input('group-select', 'value')]
)
def update_graph(grpname):
    fig = makeshotchart(all_teams_df[all_teams_df.team == grpname], col_col='ppp', range_color=[90, 120], size_col='shts_freq')
    clean_chart_format(fig)
    if len(grpname) > 3: 
        # Leaders chart size
        fig.update_layout(height=950, width=1250)
    else:
        fig.update_layout(height=550, width=1250)

    return fig

In [7]:
# Run Local Server 
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2020 17:52:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:52:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:52:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:52:13] "POST /_dash-update-component HTTP/1.1" 200 -
